In [1]:
# Imports (numpy, pandas, matplotlib, seaborn imported automatically)
from etherscan import Etherscan
from BondingCurveNexus import config

In [2]:
# activate etherscan client
eth_client = Etherscan(config.etherscan_api)

eth_price = float(eth_client.get_eth_last_price()['ethusd'])

In [3]:
eth_price

2025.6

# Investigate patterns in cover amounts and claim frequencies

## Bring in files and convert datetimes

In [4]:
# bring in csvs

# historic active cover amount
active_cover_amount = pd.read_csv('../raw_data/active_cover_amount.csv')
active_cover_amount.columns = ['date', 'USD', 'ETH']
active_cover_amount['date'] = pd.to_datetime(active_cover_amount['date']).dt.date

# claims history
claims_history = pd.read_csv('../raw_data/all_claims.csv')
claims_history['timestamp'] = pd.to_datetime(claims_history['timestamp'])

# cover history
cover_history = pd.read_csv('../raw_data/all_covers.csv')
cover_history['start_time'] = pd.to_datetime(cover_history['start_time']).dt.date

## Investigate Distribution of Daily changes in Active cover amount

In [5]:
active_cover_amount = active_cover_amount.groupby('date').max()

In [6]:
active_cover_amount['ETH_change'] = active_cover_amount['ETH'] / active_cover_amount['ETH'].shift(1) - 1

In [7]:
# from 5th day onwards - early shifts are very severe
active_cover_amount['ETH_change'].iloc[5:].describe()

count    916.000000
mean       0.007489
std        0.068623
min       -0.636676
25%       -0.010764
50%        0.000050
75%        0.015307
max        1.407936
Name: ETH_change, dtype: float64

## Investigate premium income

In [8]:
cover_history['premium_eth'] = np.where(cover_history['currency'] == 'DAI',
                                         cover_history['premium']/eth_price, cover_history['premium'])

In [9]:
# have a premium history
premium_history = cover_history.groupby('start_time')[['premium_eth']].sum().reset_index()

In [10]:
premium_history = premium_history.iloc[153:]

In [11]:
premium_history = premium_history.set_index('start_time')

In [12]:
# fill in empty dates with no cover buys
idx = pd.date_range('2020-06-01', '2022-05-16')
premium_history.reindex(idx, fill_value=0)

,premium_eth
2020-06-01,3.004682
2020-06-02,9.492877
2020-06-03,0.158140
2020-06-04,0.000000
2020-06-05,0.000000
...,...
2022-05-12,0.294067
2022-05-13,28.108227
2022-05-14,9.086183
2022-05-15,16.123214


In [13]:
premium_history.describe()

,premium_eth
count,697.000000
mean,20.275825
std,143.837127
min,0.000105
25%,1.922110
50%,5.630465
75%,13.417656
max,3687.838154


## Investigate frequency and size of accepted claims

In [14]:
# filter accepted claims only
accepted_claims = claims_history[claims_history['verdict'] == 'Accepted']

# work out days since first claim
days_since_first_claim = (pd.Timestamp.now() - accepted_claims['timestamp'].iloc[0]).days

# count number of claims and divide by number of days
avg_claim_freq = accepted_claims['verdict'].count() / days_since_first_claim

avg_claim_freq 

0.03155339805825243

In [15]:
accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',
                                         accepted_claims['amount']/eth_price, accepted_claims['amount'])

/tmp/ipykernel_615/834324826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',


In [16]:
accepted_claims

,timestamp,cover_id,block_number,verdict,claim_id,project,amount_usd,amount,currency,start_time,amount_eth
0,2020-02-18 17:01:18,152,9508333,Accepted,2,bZx v1,8050.76,4.0,ETH,2020-02-03 19:58:33,4.000000
2,2020-02-19 23:55:22,158,9516760,Accepted,5,bZx v1,2598.70,2600.0,DAI,2020-02-05 01:27:47,1.283570
3,2020-02-18 17:07:50,121,9508364,Accepted,3,bZx v1,29985.00,30000.0,DAI,2019-12-10 15:49:23,14.810427
69,2021-02-04 23:20:43,729,11792745,Accepted,72,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-24 17:36:23,6.000000
70,2021-02-05 02:47:32,278,11793668,Accepted,73,Yearn Finance (all vaults),1399.30,1400.0,DAI,2020-04-16 00:45:45,0.691153
72,2021-02-05 02:49:38,616,11793678,Accepted,74,Yearn Finance (all vaults),48304.56,24.0,ETH,2020-08-16 10:02:18,24.000000
73,2021-02-05 03:19:09,203,11793813,Accepted,75,Yearn Finance (all vaults),8995.50,9000.0,DAI,2020-02-20 04:33:53,4.443128
74,2021-02-05 17:42:46,652,11797703,Accepted,77,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-17 21:28:10,6.000000
75,2021-02-05 18:58:03,1815,11798055,Accepted,78,Yearn Finance (all vaults),20126.90,10.0,ETH,2020-09-16 23:08:00,10.000000
76,2021-02-06 07:31:02,759,11801426,Accepted,80,Yearn Finance (all vaults),100634.50,50.0,ETH,2020-08-28 06:40:35,50.000000


In [17]:
accepted_claims.amount_eth.describe()

count      26.000000
mean       60.219984
std       193.485521
min         0.691153
25%         5.202607
50%        14.810427
75%        31.815265
max      1000.000000
Name: amount_eth, dtype: float64